## Jupyter notebook for running transformer metrics

In [ ]:
from configuration_electra import ElectraConfig
from modeling_electra import ElectraModel
from modeling_electra import ElectraLayer
from transformers import ElectraTokenizerFast

import csv
from datasets import load_dataset
import json
import numpy as np
import torch

from sklearn.metrics import pairwise_distances

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
configs = []
with open("data/nas_benchmark.json", 'r') as f:
    configs = json.load(f)

In [ ]:
# Target dataset is openwebtex
dataset = load_dataset("openwebtext")
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-small-discriminator")

def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_dataset = dataset.map(encode, batched=True, num_proc=32)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
# get sample tokenized batch from dataset
dataloader = torch.utils.data.DataLoader(dataset['train'], batch_size=128)
inputs = tokenizer(next(iter(dataloader))['text'], truncation=True, padding='max_length', return_tensors="pt")

In [ ]:
# Covariance calculations for Jacobian covariance and variations
def covariance(jacobs):
    jacob = torch.transpose(jacobs, 0, 1).reshape(jacobs.size(1), -1).cpu().numpy()
    correlations = np.corrcoef(jacob)
    v, _ = np.linalg.eig(correlations)
    k = 1e-5
    return -np.sum(np.log(v + k) + 1.0 / (v + k))

In [ ]:
# Cosine calculations for Jacobian cosine and variations
def cosine(jacobs):
    jacob = torch.transpose(jacobs, 0, 1).reshape(jacobs.size(1), -1).cpu().numpy()
    norm = np.linalg.norm(jacob, axis=1)
    normed = jacob / norm[:, None]
    cosines = (-pairwise_distances(normed, metric="cosine") + 1) - np.identity(
        normed.shape[0]
    )
    summed = np.sum(np.power(np.absolute(cosines.flatten()), 1.0 / 20)) / 2
    return 1 - (1 / (pow(cosines.shape[0], 2) - cosines.shape[0]) * summed)

In [ ]:
# Synaptic Diversity metric
def synaptic_diversity(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.operation.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    if (sublayer.weight is not None) and (
                        sublayer.weight.grad is not None
                    ):
                        metric_array.append(
                            torch.abs(
                                torch.norm(sublayer.weight, "nuc")
                                * torch.norm(sublayer.weight.grad, "nuc")
                            )
                        )
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
        
    return summed.detach().item()

In [ ]:
def synaptic_diversity_normalized(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.operation.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    if (sublayer.weight is not None) and (
                        sublayer.weight.grad is not None
                    ):
                        metric_array.append(
                            torch.abs(
                                torch.norm(sublayer.weight, "nuc")
                                * torch.norm(sublayer.weight.grad, "nuc")
                            )
                        )
    
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
    summed /= len(metric_array)
    
    return summed.detach().item()

In [ ]:
# Synaptic saliency metric
def synaptic_saliency(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.intermediate.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    metric_array.append(
                        torch.abs(sublayer.weight * sublayer.weight.grad)
                    )
            for sublayer in layer.output.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    metric_array.append(
                        torch.abs(sublayer.weight * sublayer.weight.grad)
                    )
                    
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
        
    return summed.detach().item()

In [ ]:
def synaptic_saliency_normalized(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.intermediate.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    metric_array.append(
                        torch.abs(sublayer.weight * sublayer.weight.grad)
                    )
            for sublayer in layer.output.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    metric_array.append(
                        torch.abs(sublayer.weight * sublayer.weight.grad)
                    )
                    
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
    summed /= len(metric_array)
        
    return summed.detach().item()

In [ ]:
# Activation Distance metric
def activation_distance(outputs):
    metric_array = []
    for output in outputs:
        output = output[0].view(output.size(1), -1)
        x = (output > 0).float()
        K = x @ x.t()
        K2 = (1.0 - x) @ (1.0 - x.t())
        metric_array.append(K + K2)
        
    summed = torch.tensor(0.0)
    for j in range(len(outputs)):
        summed += torch.nansum(metric_array[j])
    
    return summed.detach().item()

In [ ]:
def activation_distance_normalized(outputs):
    metric_array = []
    for output in outputs:
        output = output[0].view(output.size(1), -1)
        x = (output > 0).float()
        K = x @ x.t()
        K2 = (1.0 - x) @ (1.0 - x.t())
        metric_array.append(K + K2)
        
    summed = torch.tensor(0.0)
    for j in range(len(outputs)):
        summed += torch.nansum(metric_array[j])
    summed /= len(metric_array)
    
    return summed.detach().item()

In [ ]:
def jacobian_score(model):
    jacobs = model.embeddings.position_embeddings.weight.grad.detach()
    return covariance(jacobs)

In [ ]:
def jacobian_score_cosine(model):
    jacobs = model.embeddings.position_embeddings.weight.grad.detach()
    return cosine(jacobs)

In [ ]:
def num_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [ ]:
# Attention Head Importance metric
def head_importance(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.operation.operation.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    if (sublayer.weight is not None) and (
                        sublayer.weight.grad is not None
                    ) and sublayer.weight.shape[0] >= 128:
                        metric_array.append(
                            torch.abs(sublayer.weight.data * sublayer.weight.grad)
                        )
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
        
    return summed.detach().item()

In [ ]:
def head_importance_normalized(model):
    metric_array = []
    for layer in model.modules():
        if isinstance(layer, ElectraLayer):
            for sublayer in layer.operation.operation.modules():
                if isinstance(sublayer, torch.nn.Linear):
                    if (sublayer.weight is not None) and (
                        sublayer.weight.grad is not None
                    ) and sublayer.weight.shape[0] >= 128:
                        metric_array.append(
                            torch.abs(sublayer.weight.data * sublayer.weight.grad)
                        )
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
    summed /= len(metric_array)
        
    return summed.detach().item()

In [ ]:
# Attention Confidence metric (for both head and softmax)
def attention_condfidence(outputs):
    metric_array = []
    for output in outputs:
        metric_array.append(torch.mean(torch.max(output, 1)[0]))
    
    summed = torch.tensor(0.0)
    for j in range(len(outputs)):
        summed += torch.nansum(metric_array[j])
        
    return summed.detach().item()

In [ ]:
def attention_condfidence_normalized(outputs):
    metric_array = []
    for output in outputs:
        metric_array.append(torch.mean(torch.max(output, 1)[0]))
    
    summed = torch.tensor(0.0)
    for j in range(len(metric_array)):
        summed += torch.nansum(metric_array[j])
    summed /= len(metric_array)
    
    return summed.detach().item()

In [ ]:
# Run metrics on all model in benchmark
with open("BERT_results_activation.csv", "a") as f:
    writer = csv.writer(f)
    
    header = ["ID",
              "GLUE Score",
              "Synaptic Diversity",
              "Synaptic Diversity Normalized",
              "Synaptic Saliency",
              "Synaptic Saliency Normalized",
              "Activation Distance",
              "Activation Distance Normalized",
              "Jacobian Score",
              "Jacobian Score Normalized",
              "Number of Parameters",
              "Head Importance",
              "Head Importance Normalized",
              "Head Confidence",
              "Head Confidence Normalized",
              "Head Softmax Confidence",
              "Head Softmax Confidence Normalized",
             ]
    writer.writerow(header)
    f.flush()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    for i in range(500):
        np.random.seed(0)
        torch.manual_seed(0)

        nas_config = configs[i]["hparams"]["model_hparam_overrides"]["nas_config"]

        config = ElectraConfig(
            nas_config=nas_config, num_hidden_layers=len(nas_config["encoder_layers"]), output_hidden_states=True
        )
        model = ElectraModel(config)
        model.to(device)
        inputs.to(device)
        
        # Hooks to get outputs at different layers
        activation_outputs = []
        def activation_hook(module, input, output):
            activation_outputs.append(output)
        for layer in model.modules():
            if isinstance(layer, ElectraLayer):
                sublayer = layer.intermediate.intermediate_act_fn.register_forward_hook(activation_hook)

        head_outputs = []
        def head_hook(module, input, output):
            head_outputs.append(output)

        # Initialize hooks
        for layer in model.modules():
            if isinstance(layer, ElectraLayer):
                sublayer = layer.operation.operation
                if hasattr(sublayer, 'query'):
                    sublayer.query.register_forward_hook(head_hook)
                if hasattr(sublayer, 'key'):
                    sublayer.key.register_forward_hook(head_hook)
                if hasattr(sublayer, 'value'):
                    sublayer.value.register_forward_hook(head_hook)
                if hasattr(sublayer, 'input'):
                    sublayer.input.register_forward_hook(head_hook)
                if hasattr(sublayer, 'weight'):
                    sublayer.weight.register_forward_hook(head_hook)

        softmax_outputs = []
        def softmax_hook(module, input, output):
            softmax_outputs.append(output)

        for layer in model.modules():
            if isinstance(layer, ElectraLayer):
                sublayer = layer.operation.operation
                if hasattr(sublayer, 'softmax'):
                    sublayer.softmax.register_forward_hook(softmax_hook)

        # Run gradient with respect to ones
        model.zero_grad()
        output = model(**inputs).last_hidden_state
        output.backward(torch.ones_like(output))

        row = [configs[i]["id"],
               configs[i]["scores"]["glue"],
               synaptic_diversity(model),
               synaptic_diversity_normalized(model),
               synaptic_saliency(model),
               synaptic_saliency_normalized(model),
               activation_distance(activation_outputs),
               activation_distance_normalized(activation_outputs),
               jacobian_score(model),
               jacobian_score_cosine(model),
               num_parameters(model),
               head_importance(model),
               head_importance_normalized(model),
               attention_condfidence(head_outputs),
               attention_condfidence_normalized(head_outputs),
               attention_condfidence(softmax_outputs),
               attention_condfidence_normalized(softmax_outputs),
              ]
        
        writer.writerow(row)
        f.flush()

        print(str(configs[i]["id"]))

In [ ]:
ablation_models = [285,
116,
280,
337,
464,
166,
153,
157,
330,
164,]

In [ ]:
# Run metrics on ablation study of different minibatch inputs
with open("BERT_batch_ablation.csv", "a") as f:
    writer = csv.writer(f)
    
    header = ["ID",
              "GLUE Score",
              "Synaptic Diversity",
              "Synaptic Diversity Normalized",
              "Synaptic Saliency",
              "Synaptic Saliency Normalized",
              "Activation Distance",
              "Activation Distance Normalized",
              "Jacobian Score",
              "Jacobian Score Normalized",
              "Number of Parameters",
              "Head Importance",
              "Head Importance Normalized",
              "Head Confidence",
              "Head Confidence Normalized",
              "Head Softmax Confidence",
              "Head Softmax Confidence Normalized",
             ]
    writer.writerow(header)
    f.flush()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    for i in ablation_models:
        nas_config = configs[i]["hparams"]["model_hparam_overrides"]["nas_config"]

        config = ElectraConfig(
            nas_config=nas_config, num_hidden_layers=len(nas_config["encoder_layers"]), output_hidden_states=True
        )
        model = ElectraModel(config)
        model.to(device)
        inputs.to(device)
        
        iterator = iter(dataloader)
        
        for j in range(10):
            inputs = tokenizer(next(iterator)['text'], truncation=True, padding='max_length', return_tensors="pt")
            inputs.to(device)

            np.random.seed(0)
            torch.manual_seed(0)

            activation_outputs = []
            def activation_hook(module, input, output):
                activation_outputs.append(output)
            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.intermediate.intermediate_act_fn.register_forward_hook(activation_hook)

            head_outputs = []
            def head_hook(module, input, output):
                head_outputs.append(output)

            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.operation.operation
                    if hasattr(sublayer, 'query'):
                        sublayer.query.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'key'):
                        sublayer.key.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'value'):
                        sublayer.value.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'input'):
                        sublayer.input.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'weight'):
                        sublayer.weight.register_forward_hook(head_hook)

            softmax_outputs = []
            def softmax_hook(module, input, output):
                softmax_outputs.append(output)

            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.operation.operation
                    if hasattr(sublayer, 'softmax'):
                        sublayer.softmax.register_forward_hook(softmax_hook)


            model.zero_grad()
            output = model(**inputs).last_hidden_state
            output.backward(torch.ones_like(output))

            row = [configs[i]["id"],
                   configs[i]["scores"]["glue"],
                   synaptic_diversity(model),
                   synaptic_diversity_normalized(model),
                   synaptic_saliency(model),
                   synaptic_saliency_normalized(model),
                   activation_distance(activation_outputs),
                   activation_distance_normalized(activation_outputs),
                   jacobian_score(model),
                   jacobian_score_cosine(model),
                   num_parameters(model),
                   head_importance(model),
                   head_importance_normalized(model),
                   attention_condfidence(head_outputs),
                   attention_condfidence_normalized(head_outputs),
                   attention_condfidence(softmax_outputs),
                   attention_condfidence_normalized(softmax_outputs),
                  ]

            writer.writerow(row)
            f.flush()

            print(str(configs[i]["id"]))

In [ ]:
# Run metrics on ablation study of different initialization states
with open("BERT_initialization_ablation.csv", "a") as f:
    writer = csv.writer(f)
    
    header = ["ID",
              "GLUE Score",
              "Synaptic Diversity",
              "Synaptic Diversity Normalized",
              "Synaptic Saliency",
              "Synaptic Saliency Normalized",
              "Activation Distance",
              "Activation Distance Normalized",
              "Jacobian Score",
              "Jacobian Score Normalized",
              "Number of Parameters",
              "Head Importance",
              "Head Importance Normalized",
              "Head Confidence",
              "Head Confidence Normalized",
              "Head Softmax Confidence",
              "Head Softmax Confidence Normalized",
             ]
    writer.writerow(header)
    f.flush()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    for i in ablation_models:
        nas_config = configs[i]["hparams"]["model_hparam_overrides"]["nas_config"]

        config = ElectraConfig(
            nas_config=nas_config, num_hidden_layers=len(nas_config["encoder_layers"]), output_hidden_states=True
        )
        model = ElectraModel(config)
        model.to(device)
        inputs.to(device)
        
        for j in range(10):
            np.random.seed(j)
            torch.manual_seed(j)

            activation_outputs = []
            def activation_hook(module, input, output):
                activation_outputs.append(output)
            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.intermediate.intermediate_act_fn.register_forward_hook(activation_hook)

            head_outputs = []
            def head_hook(module, input, output):
                head_outputs.append(output)

            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.operation.operation
                    if hasattr(sublayer, 'query'):
                        sublayer.query.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'key'):
                        sublayer.key.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'value'):
                        sublayer.value.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'input'):
                        sublayer.input.register_forward_hook(head_hook)
                    if hasattr(sublayer, 'weight'):
                        sublayer.weight.register_forward_hook(head_hook)

            softmax_outputs = []
            def softmax_hook(module, input, output):
                softmax_outputs.append(output)

            for layer in model.modules():
                if isinstance(layer, ElectraLayer):
                    sublayer = layer.operation.operation
                    if hasattr(sublayer, 'softmax'):
                        sublayer.softmax.register_forward_hook(softmax_hook)


            model.zero_grad()
            output = model(**inputs).last_hidden_state
            output.backward(torch.ones_like(output))

            row = [configs[i]["id"],
                   configs[i]["scores"]["glue"],
                   synaptic_diversity(model),
                   synaptic_diversity_normalized(model),
                   synaptic_saliency(model),
                   synaptic_saliency_normalized(model),
                   activation_distance(activation_outputs),
                   activation_distance_normalized(activation_outputs),
                   jacobian_score(model),
                   jacobian_score_cosine(model),
                   num_parameters(model),
                   head_importance(model),
                   head_importance_normalized(model),
                   attention_condfidence(head_outputs),
                   attention_condfidence_normalized(head_outputs),
                   attention_condfidence(softmax_outputs),
                   attention_condfidence_normalized(softmax_outputs),
                  ]

            writer.writerow(row)
            f.flush()

            print(str(configs[i]["id"]))